<h1>Universidade Federal de Lavras </h1>
<h3>Trabalho Prático - GCC218</h3>
<h2>04 Outubro 2019 / Semestre 02</h2>

<p>Alunos:</p>
<p>Fábio Tavares</p>
<p>James Yuri </p>
<p>João Vitor</p>
 
<h1>Introdução</h1> 
<p>O planejamento de horários em instituições de ensino constitui um desafio para diretores e coordenadores pedagógicos. De fato, trata-se de uma tarefa passível a erros, devido ao seu caráter combinatório, e que exige demasiado tempo de elaboração. O problema consiste em designar horários a turmas já pré-estabelecidas. Neste trabalho a técnica de solução fundamenta-se na Teoria dos Grafos, mais precisamente no método de Coloração de vertice realizado por meio do algoritmo D-Satur com algumas adaptações.</p> 
<p>O problema apresentado caracteriza-se como Timetabling Problem(Problema de Horários) que, quando aplicado ao contexto educacional, consiste em agendar uma série de encontros entre alunos, professores e instituições de ensino como, por exemplo, aulas, exames, cursos, entre outras atividades inerentes a este contexto, satisfazendo a um conjunto de restrições (WERRA, 1985; SCHAERF, 1999).</p>
<p>O problema apresentado faz parte da classe de problemas NP-Difícil, com isso nao consegue garantir que o problema seja resolvido em tem polinomal</p>



<h1>Modelagem</h1>
<p>Na modelagem, os professores, horários e disciplinas foram representados como vértices
do grafo bipartido e os requisitos como arestas ou atributos de arestas. Para exemplificar, um
interesse de um professor em lecionar uma disciplina foi representado por uma aresta unindo os
vertices do professor e da disciplina. Os vértices de horários foram usados como vértices de ligação
entre os vértices de professores e disciplina, com o objetivo de criar um gargálo no grafo, garantindo
a restrição de ocupação na agenda do professor. As preferências foram associadas a custos nas
arestas que representam o interesse de um professor a uma disciplina.<p>

![ModelagemGrafos.png](attachment:ModelagemGrafos.png)

<h1> MÉTODO DE SOLUÇÃO</h1>
<p>Um grafo é um par G=(V ,E) , onde V é um conjunto finito e E um conjunto de subconjuntos de dois elementos de V . Os elementos de V são chamados vértices do grafo e os elementos de E são chamados de arestas do grafo. O método utilizado para solucionar o problema fundamenta-se na Teoria dos Grafos,mais precisamente na Coloração de vértices também conhecida como Coloração de Grafos.</p>

<p>Pensando no fato que cada escola tem horários de aula diferentes pegamos as informações dos horário da escola pela planilha que chamaremos de "Configurações". Depois de pegar as configurações da escola, colocamos em uma lista juntamente com a quantidade de dias da semana, forma uma lista com os horários, no formato (Dia da semana) - (Hora da aula).</p>

In [1]:

#Abrindo a planilha
# path = "/home/desenvolvedor2/Documentos/Python/horario_grafos/public/files/Escola_A.xlsx"
path = "../public/files/Escola_A.xlsx"
wbObj = openpyxl.load_workbook(path)

schollData = wbObj['Dados']
settings = wbObj['Configuracoes']
classRestriction = wbObj['Restricoes Turma']
teacherRestriction = wbObj['Restricao']

NameError: name 'openpyxl' is not defined

<p>Criação dos horarios apartir da coluna configurações do arquivo</p>

In [15]:
days_week = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
schedules = []

for day in days_week:
    jump = True
    for i, row in enumerate(settings.rows):
        if(i > 0):
            schedules.append("{} - {}".format(day, row[0].value))

<p>Para tratar as restrições de turma e professor, foi adicionado uma lista em cada vértice, contendo os índices do horário, referente a lista gerada acima, que o professor e a turma não podem ter.<p>

<p>Instanciando e preenchendo lista de professores e turmas e adicionado as restrições da turma apartir da coluna Restrições turma que está no arquivo em uma lista e dicionário, respectivamente</p>

In [24]:
# instanciando a lista de restrição de cada turma e de cada professor
classes = []
teachers = []
#Colocando o os professores e turmas em uma lista
for i, row in enumerate(schollData.rows):
    if(i > 0):
        if (not row[0].value in classes):
            classes.append(row[0].value)

        if (not row[2].value in classes):
            teachers.append(row[2].value)
allClassRestriction = []
for i in range(len(classes)):
    allClassRestriction.append([])

for i, row in enumerate(classRestriction.rows):
    if(i > 0):
        scheduleRestriction = "{} - {}".format(row[2].value, row[1].value)
        allClassRestriction[int(row[0].value) - 1].append(schedules.index(scheduleRestriction))

<p>Adicionado as restrições dos professores apartir da coluna Restrições turma que está no arquivo em uma lista e dicionário, respectivamente</p>

In [25]:
allTeacherRestriction = {}
for teacher in teachers:
    allTeacherRestriction[teacher] = []

for i, row in enumerate(teacherRestriction.rows):
    if(i > 0):
        scheduleRestriction = "{} - {}".format(row[2].value, row[1].value)
        allTeacherRestriction[row[0].value].append(schedules.index(scheduleRestriction))

<h2>Lista dos vertices</h2>
<p>Criação dos vértices que contém as informações da aula e adicionando todos os vértices em uma lista, tal vertice contém os seguintes atributos: identificador, matéria, turma, professor, lista com restrição da turma e lista com restrição do professor</p>

In [26]:
import openpyxl
class VertexData(object):
    def __init__(self, vertexDataId, subject, schoolClass, teacher, classRestriction, teacherRestriction):
        self.vertexDataId, self.subject, self.schoolClass, self.teacher = vertexDataId, subject, schoolClass, teacher
        self.classRestriction = classRestriction
        self.teacherRestriction = teacherRestriction
        self.adjacents, self.schedule_id, self.saturation = [], None, -1

<p>Inicializando a lista de vértice e uma lista dos identificadores de cada vertice não coloridos, e n para 0, n no final do algoritimo será o numero de vértices e durante a execução do algoritimo será o id do vertice</p>

In [27]:
vertex = []
vertexListId = []
n = 0
# Criando os vértices e adicionando todos os vértices em uma lista
# Cria um vértice com os um identificador, matéria, turma, professor,
# lista com restrição da turma e lista com restrição do professor.
# De acordo com a quantidade de aulas desta matéria, com o mesmo professor e turma
# e criado um vértice com as mesmas informações
for i, row in enumerate(schollData.rows):
    if(i > 0):
        numberClasses = int(row[3].value)
        for i in range(numberClasses):
            vertexListId.append(n)
            vertexData = VertexData(n, row[0].value, row[1].value, row[2].value, allClassRestriction[int(row[1].value) - 1], allTeacherRestriction[row[2].value])
            vertex.append(vertexData)
            n += 1

<h1>Criando as ligações do vértice</h1>
<p>Adiciona arestas ligando os vértice que possuem a mesma sala de aula e o mesmo professor com identificadores diferentes.</p>

In [31]:
# Criando as arestas do grafo.
# Adiciona os vértices vizinhos em uma lista contendo os vértices adjacente do mesmo.
for v1 in vertex:
    for v2 in vertex:
        if(v1.vertexDataId != v2.vertexDataId and (v1.schoolClass == v2.schoolClass or v1.teacher == v2.teacher)):
            v1.adjacents.append(v2)
            v2.adjacents.append(v1)

<h1>Coloração</h1>
<p>O algoritmo conhecido como algoritmo DSATUR foi tirado como base para a implementação deste trabalho.</p>

In [30]:
# Algoritmo de coloração feito com base no algoritmo de Dsatur
colors = []
while(len(vertexListId) > 0):
    # Primeiro realiza uma busca para pegar o vértice com o maior número de saturação
    # ou caso o maior valor de saturação seja repetido com o maior grau.
    maxVertex = vertex[vertexListId[0]]
    for index in vertexListId:
        if(vertex[index].saturation > maxVertex.saturation or (vertex[index].saturation == maxVertex.saturation
            and len(vertex[index].adjacents) > len(maxVertex.adjacents))):
            maxVertex = vertex[index]
    # Colorindo o vértice com a menor cor possível, levando em consideração a restrição
    # de horário da turma e do professor e que nenhum dos seus adjacentes tenha a mesma
    # cor.
    color = 0
    colorValidate = False
    while(not colorValidate):
        colorValidate = True
        if(color in maxVertex.classRestriction or color in maxVertex.teacherRestriction):
            colorValidate = False
            color += 1
        else:
            for adjacent in maxVertex.adjacents:
                if(adjacent.schedule_id == color):
                    colorValidate = False
                    color += 1
    maxVertex.schedule_id = color
    # Aumentando o grau de saturação dos vértices adjacentes ao vértice pintado
    for adjacent in maxVertex.adjacents:
        adjacent.saturation += 1
    # Adicionando em uma lista as cores utilizadas na coloração dos vértices
    if(not color in colors):
        colors.append(color)
    # Remove da lista de vértices ainda não coloridos o vértice que foi colorido
    vertexListId.remove(maxVertex.vertexDataId)

<h2>Encontrando a cor possível</h2>
<p>Na linha 1 inicializamos a lista de cores como vázia, na linha 2 ocorre um laço que corresponde ao numero de vezes. Da linha 5 a 9 é realizada uma busca para extrair o vértice com o maior grau de saturação. Na linha 13 inicializa a cor que será usada como 0, na linha 14 setá a cor como não válida, na linha 15 ocorre um laço que irá ocorrer até achar uma cor válida, na linha 16 seta a cor para uma cor válida, na linha 17 verifica se a cor está na lista de restrição de turma e professores, caso a cor estiver em pelo menos uma na linha 18 a cor passa a não ser válida e na linha 19 incrementa a cor, caso não estiver na linha 21 irá ocorrer um laço por toda lista de adjacência do vértice de maior grau, na linha 22 verifica se o horário é igual a cor do vértice adjacênte ao vertice de maior grau, caso ele seja significa que um dos adjacêntes do vértice maior grau já ocupa o horário correspondente a cor, então na linha 23 a cor passa a ser não válida e a cor é incrementado.</p>

<h2>Atribuindo horarios</h2>
<p>Na linha 24 atribui o horario para o vertice de maior grau com base no número da cor correspondentes das linhas 1 - 23, na linha 25 ocorre um laço para todos os adjacêntes do vértice de maior grau, na linha 26 é incrementando a saturação de cada um deles.</p>
<h2>Lista de cores</h2>
<p>Na linha 27 verifica se a cor está na lista de cores, caso não esteja, a cor é adicionada na lista de cores. Na linha 28</p>
<p>Linha 30 o vértice de maior grau é removido da lista de vértice não coloridos.<p>


<h1>Algoritimo de melhoramento</h1>

In [ ]:
<p></p>

<h2>Lista de cores utilizadas</h2>
<p>Na linha 28 é verificado de a cor já está na lista de cores, caso não esteja adiciona a cor na lista na linha 29. Na linha 31 o vertice é removido dos vertices não coloridos</p>

<h1>Lista com cores usadas pelos vértices</h1>

In [ ]:
# Gera uma lista com a cores que estão sendo usadas pelos vértices
colors = []
for v in vertex:
    if(not v.schedule_id in colors):
        colors.append(v.schedule_id)

<p>imprime o número de cores</p>

In [ ]:
print("Número de cores:", len(colors))

<h1>Horários</h1>

In [ ]:
# Exibe no tela os vértices, separados pelo horário (cor)
notColoring = []
for i, color in enumerate(colors):
    if(i < len(schedules)):
        print(schedules[i])
        for v in vertex:
            if(v.schedule_id == color) :
                print(v.schedule_id, " / Matéria", v.subject, "/ ", v.teacher, "/ Turma", v.schoolClass)
    else:
        notColoring.append(v)


In [ ]:
<h1>Complexidade</h1>

<h1>Tecnologia utilizadas:</h1>
<p><strong>Python: </strong>é uma linguagem de programação que permite trabalhar mais rapidamente e integrar seus sistemas com mais eficiência.</p>
<p><strong>Openpyxl: </strong> é uma biblioteca Python para ler/gravar arquivos xlsx/xlsm/xltx/xltm do Excel.</p>
